<a href="https://colab.research.google.com/github/yashch1511/Lunar-AI-model/blob/main/Deep_Q_Learning_for_Lunar_Landing_Partial_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Q-Learning for Lunar Landing

## Part 0 - Installing the required packages and importing the libraries

### Installing Gymnasium

In [ ]:
!pip install gymnasium
!pip install "gymnasium[atari, accept-rom-license]"
!apt-get install -y swig
!pip install gymnasium[box2d]

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
swig is already the newest version (4.0.2-1ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


### Importing the libraries

In [ ]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.autograd as autograd
from torch.autograd import Variable
from collections import deque, namedtuple

## Part 1 - Building the AI

### Creating the architecture of the Neural Network

In [ ]:
class Network(nn.Module):
  def __init__(self, state_size, action_size,seed=42) -> None:
      super(Network,self).__init__()
      self.seed = torch.manual_seed(seed)
      self.fc1 = nn.Linear(state_size,64)
      self.fc2 = nn.Linear(64,64)
      self.fc3 = nn.Linear(64,action_size)


  def forward(self,state):
      x = F.relu(self.fc1(state))
      x = F.relu(self.fc2(x))
      return self.fc3(x)

## Part 2 - Training the AI

### Setting up the environment

In [ ]:
import gymnasium as gym
env = gym.make('LunarLander-v3') # The Lunar Lander environment was upgraded to v3
state_shape = env.observation_space.shape
state_size = env.observation_space.shape[0]
number_actions = env.action_space.n
print('State shape: ', state_shape)
print('State size: ', state_size)
print('Number of actions: ', number_actions)

State shape:  (8,)
State size:  8
Number of actions:  4


### Initializing the hyperparameters

In [ ]:
learning_rate=5e-4
minibatch_size=100
gamma=0.99
replay_memory_size=100000
interpolation_factor=0.001

### Implementing Experience Replay

In [ ]:
class ReplayMemory(object):
    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []

    def push(self, event):
        self.memory.append(event)
        if len(self.memory) > self.capacity:
            del self.memory[0]

    def sample(self,batch_size):
        experiences=random.sample(self.memory,k=batch_size)
        states=torch.from_numpy(np.vstack([e[0] for e in experiences if e is not None])).float()
        actions=torch.from_numpy(np.vstack([e[1] for e in experiences if e is not None])).long()
        rewards=torch.from_numpy(np.vstack([e[2] for e in experiences if e is not None])).float()
        next_states=torch.from_numpy(np.vstack([e[3] for e in experiences if e is not None])).float()
        dones=torch.from_numpy(np.vstack([e[4] for e in experiences if e is not None]).astype(np.uint8)).float()
        return (states,actions,rewards,next_states,dones)

    def __len__(self):
        return len(self.memory)

### Implementing the DQN class

In [ ]:
class Agent():
    def __init__(self,state_size,action_size):
        self.state_size=state_size
        self.action_size=action_size
        self.local_network=Network(self.state_size,self.action_size)
        self.target_network=Network(self.state_size,self.action_size)
        self.optimizer=optim.Adam(self.local_network.parameters(),lr=learning_rate)
        self.memory=ReplayMemory(replay_memory_size)
        self.t_step=0

    def step(self,state,action,reward,next_state,done):
        self.memory.push((state,action,reward,next_state,done))
        self.t_step=(self.t_step+1)%4
        if self.t_step==0:
            if len(self.memory)>minibatch_size:
                experiences=self.memory.sample(minibatch_size)
                self.learn(experiences,gamma)

    def act(self,state,epsilon=0.):
        state=torch.from_numpy(state).float().unsqueeze(0)
        self.local_network.eval()
        with torch.no_grad():
            action_values=self.local_network(state)
        self.local_network.train()
        if random.random()>epsilon:
            return np.argmax(action_values.cpu().data.numpy())
        else:
            return random.choice(np.arange(self.action_size))


    def learn(self,experiences,gamma):
        states,actions,rewards,next_states,dones=experiences
        Q_targets_next=self.target_network(next_states).detach().max(1)[0].unsqueeze(1)
        Q_targets=rewards+gamma*Q_targets_next*(1-dones)
        Q_expected=self.local_network(states).gather(1,actions)
        loss=F.mse_loss(Q_expected,Q_targets)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        self.soft_update(self.local_network,self.target_network,interpolation_factor)


    def soft_update(self,local_model,target_model,interpolation_factor):
        for target_param,local_param in zip(target_model.parameters(),local_model.parameters()):
            target_param.data.copy_(interpolation_factor*local_param.data+( 1.0-interpolation_factor)*target_param.data)



### Initializing the DQN agent

In [ ]:
agent=Agent(state_size,number_actions)

### Training the DQN agent

In [ ]:
number_episodes=2000
max_t=1000
epsilon_start=1.0
epsilon_end=0.01
epsilon_decay=0.995
epsiolon=epsilon_start
scores_window=deque(maxlen=100)

for i_episode in range(1,number_episodes+1):
    state,_=env.reset()
    score=0
    for t in range(max_t):
        action=agent.act(state,epsiolon)
        next_state,reward,done,_ ,_=env.step(action)
        agent.step(state,action,reward,next_state,done)
        state=next_state
        score+=reward
        if done:
            break
    scores_window.append(score)
    epsiolon=max(epsilon_end,epsilon_decay*epsiolon)
    print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)), end="")
    if i_episode % 100 == 0:
        print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))
        torch.save(agent.local_network.state_dict(), 'checkpoint.pth')
        if np.mean(scores_window)>=200.0:
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode-100, np.mean(scores_window)))
            torch.save(agent.local_network.state_dict(), 'checkpoint.pth')
            break

Episode 100	Average Score: -169.20
Episode 200	Average Score: -112.62
Episode 300	Average Score: -27.00
Episode 400	Average Score: 49.97
Episode 500	Average Score: 104.06
Episode 600	Average Score: 141.36
Episode 700	Average Score: 146.33
Episode 800	Average Score: 159.51
Episode 900	Average Score: 204.99

Environment solved in 800 episodes!	Average Score: 204.99


## Part 3 - Visualizing the results

In [ ]:
import glob
import io
import base64
import imageio
from IPython.display import HTML, display

def show_video_of_model(agent, env_name):
    env = gym.make(env_name, render_mode='rgb_array')
    state, _ = env.reset()
    done = False
    frames = []
    while not done:
        frame = env.render()
        frames.append(frame)
        action = agent.act(state)
        state, reward, done, _, _ = env.step(action.item())
    env.close()
    imageio.mimsave('video.mp4', frames, fps=30)

show_video_of_model(agent, 'LunarLander-v3')

def show_video():
    mp4list = glob.glob('*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        display(HTML(data='''<video alt="test" autoplay
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
    else:
        print("Could not find video")

show_video()